In [1]:
import math
import pandas as pd 
import numpy as np


data = pd.read_csv('cat2.csv')


# print(data.head())


#features
x = data.drop(['index', 'galex_objid', 'sdss_objid', 'class', 'spectrometric_redshift', 'pred'], axis = 1) 
# print(x.head())


#class
y = data['class']
# print(y.head())

In [2]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))



#fix scientifci notations to normal floats
# def scitofloat(x):
#     return x

In [3]:
#normalise the values of x

outercnt = 0
for index, row in x.iterrows():
    innercnt = 0
    for column in x:
#         x.at[outercnt, column] = scitofloat(x.iloc[outercnt, innercnt])
        x.at[outercnt, column] = sigmoid(x.iloc[outercnt, innercnt])
#         print(outercnt, column, '\t\t', x.iloc[outercnt, innercnt])
        innercnt += 1
    outercnt += 1    

#     print('\n\n')

print(x.head())

   Unnamed: 0    u    g    r    i    z  extinction_u  extinction_g  \
0           0  1.0  1.0  1.0  1.0  1.0      0.520923      0.516307   
1           0  1.0  1.0  1.0  1.0  1.0      0.525265      0.519693   
2           1  1.0  1.0  1.0  1.0  1.0      0.520914      0.516300   
3           1  1.0  1.0  1.0  1.0  1.0      0.517817      0.513885   
4           1  1.0  1.0  1.0  1.0  1.0      0.514838      0.511563   

   extinction_r  extinction_i  ...       g-z       r-i       r-z       i-z  \
0      0.511283      0.508385  ...  0.384160  0.450753  0.425372  0.474241   
1      0.513627      0.510127  ...  0.612969  0.601924  0.597798  0.495702   
2      0.511278      0.508381  ...  0.502978  0.467984  0.511389  0.543341   
3      0.509607      0.507139  ...  0.965257  0.766892  0.889908  0.710736   
4      0.508000      0.505945  ...  0.910999  0.621933  0.709023  0.596976   

    fuv-nuv     fuv-u     fuv-g     fuv-r     fuv-i     fuv-z  
0  0.290620  0.154052  0.150864  0.174125  0.2

In [5]:
def info_dataset(data, verbose=True):
    label1, label2 = 0, 0
    data_size = len(data)
    for index, row in data.iterrows():
        label = row['class']
#         print(label)
        if label == 1:
            label1 += 1
        else:
            label2 += 1
    if verbose:
        print('Total of samples: %d' % data_size)
        print('Total label 1: %d' % label1)
        print('Total label 0: %d' % label2)
    return [len(data), label1, label2]

In [6]:
p = 0.6
_, label1, label2 = info_dataset(data,False)

info_dataset(data)

Total of samples: 3646
Total label 1: 3229
Total label 0: 417


[3646, 3229, 417]

In [7]:
#get training set and test set

from sklearn.model_selection import train_test_split

xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2, random_state = 15)

train_set = pd.concat([xTrain, yTrain], axis=1, sort=False)
test_set = pd.concat([xTest, yTest], axis=1, sort=False)


print(train_set.iloc[0:4])
# print(train_set.iloc[0][1])

      Unnamed: 0    u    g    r    i    z  extinction_u  extinction_g  \
1502           1  1.0  1.0  1.0  1.0  1.0      0.539908      0.531122   
1845           1  1.0  1.0  1.0  1.0  1.0      0.532798      0.525570   
762            1  1.0  1.0  1.0  1.0  1.0      0.522082      0.517210   
2756           1  1.0  1.0  1.0  1.0  1.0      0.601229      0.579307   

      extinction_r  extinction_i  ...       r-i       r-z       i-z   fuv-nuv  \
1502      0.521544      0.516014  ...  0.496376  0.560282  0.563850  0.510352   
1845      0.517697      0.513153  ...  0.534418  0.685624  0.655171  0.485502   
762       0.511909      0.508850  ...  0.505141  0.526056  0.520926  0.372240   
2756      0.555106      0.541025  ...  0.516381  0.616212  0.600597  0.247593   

         fuv-u     fuv-g     fuv-r     fuv-i     fuv-z  class  
1502  0.471202  0.315107  0.279490  0.282418  0.233384      1  
1845  0.227260  0.142308  0.121282  0.107337  0.059520      1  
762   0.166430  0.060207  0.049899  

In [8]:
def euclidian_dist(p1, p2):
    dim, sum_ = len(p1), 0
    for index in range(dim - 1):
        sum_ += math.pow(p1[index] - p2[index], 2)
    return math.sqrt(sum_)

In [11]:
def knn(train_set, new_sample, K):

#     print('train set', train_set, type(train_set))
#     print('new sample', new_sample, type(new_sample))
    dists, train_size = {}, len(train_set)

    for i in range(train_size):
#         print(train_set.iloc[i-1].values.tolist())
#         print(new_sample.values.tolist()[0])
#         print('\n\n')
        d = euclidian_dist(train_set.iloc[i-1].values.tolist(), new_sample.values.tolist()[0])
        dists[i] = d
        
#     print('done')
    
    k_neighbors = sorted(dists, key=dists.get)[:K]
    
    qty_label1, qty_label2 = 0, 0
    
    for index in k_neighbors:
#         print('index, ', index)
#         print(train_set.iloc[index])
        
    
        #fix this
        if train_set.iloc[index].values.tolist()[-1] == 1:
            qty_label1 += 1
        else:
            qty_label2 += 1
        
            
    if qty_label1 > qty_label2:
        return 1
    else:
        return 0

In [15]:
correct, K = 0, 5

count, tp, fp, tn, fn = 0, 0, 0, 0, 0


for index, sample in test_set.iterrows():
    print(count)
    count +=1
    sample = pd.DataFrame(sample)
    sample = sample.T

    label = knn(train_set, sample, K)

    if label == sample.iloc[0].values.tolist()[-1]:
        correct += 1
    
    print('predicted, actual', label, sample.iloc[0].values.tolist()[-1])
    if label == 1 and sample.iloc[0].values.tolist()[-1] == 1:
        tp += 1
    if label == 1 and sample.iloc[0].values.tolist()[-1] == 0:
        fp += 1 
    if label == 0 and sample.iloc[0].values.tolist()[-1] == 1:
        fn += 1    
    if label == 0 and sample.iloc[0].values.tolist()[-1] == 0:
        tn += 1    

print('total: ', count)
print('correct: ', correct)
print('\n\n')

matrix_list = np.array([[tp, fp], [fn, tn]])

# matrix = pd.DataFrame({'Actual +ve': matrix_list[:, 0], 'Actual -ve': matrix_list[:, 1]})
matrix = pd.DataFrame(matrix_list, index=['Pred +ve', 'Pred -ve'], columns=['Actual +ve', 'Actual -ve'])



print(matrix)

0
index,  1742
index,  1856
index,  18
predicted, actual 1 1.0
1
index,  999
index,  2727
index,  1793
predicted, actual 1 1.0
2
index,  2316
index,  1644
index,  572
predicted, actual 1 1.0
3
index,  850
index,  488
index,  71
predicted, actual 1 1.0
4
index,  2608
index,  2599
index,  2112
predicted, actual 1 1.0
5
index,  1121
index,  728
index,  387
predicted, actual 1 1.0
6
index,  804
index,  1467
index,  2277
predicted, actual 1 1.0
7
index,  2061
index,  2246
index,  2844
predicted, actual 1 1.0
8
index,  2542
index,  2158
index,  1152
predicted, actual 1 1.0
9
index,  2169
index,  1380
index,  12
predicted, actual 1 1.0
10
index,  1618
index,  283
index,  2520
predicted, actual 1 1.0
11
index,  632
index,  2024
index,  2749
predicted, actual 1 0.0
12
index,  2141
index,  1434
index,  775
predicted, actual 1 1.0
13
index,  1824
index,  2515
index,  1124
predicted, actual 1 1.0
14
index,  473
index,  2258
index,  1678
predicted, actual 1 1.0
15
index,  2449
index,  2398
index,  

index,  1667
index,  2457
index,  218
predicted, actual 1 1.0
128
index,  2142
index,  779
index,  2010
predicted, actual 1 1.0
129
index,  2042
index,  1131
index,  1054
predicted, actual 0 1.0
130
index,  1929
index,  1991
index,  2835
predicted, actual 1 0.0
131
index,  552
index,  2748
index,  1456
predicted, actual 0 1.0
132
index,  2796
index,  1637
index,  1111
predicted, actual 1 1.0
133
index,  2280
index,  209
index,  1098
predicted, actual 1 1.0
134
index,  1277
index,  268
index,  1265
predicted, actual 1 1.0
135
index,  2349
index,  2810
index,  722
predicted, actual 0 1.0
136
index,  165
index,  233
index,  1477
predicted, actual 1 1.0
137
index,  1511
index,  890
index,  2846
predicted, actual 1 1.0
138
index,  484
index,  960
index,  2794
predicted, actual 1 1.0
139
index,  1209
index,  409
index,  2723
predicted, actual 1 1.0
140
index,  1136
index,  97
index,  2865
predicted, actual 1 1.0
141
index,  939
index,  52
index,  2370
predicted, actual 1 1.0
142
index,  1982

index,  2501
index,  1612
index,  1704
predicted, actual 1 1.0
253
index,  1933
index,  2518
index,  1526
predicted, actual 1 1.0
254
index,  1303
index,  2343
index,  1717
predicted, actual 1 1.0
255
index,  1541
index,  330
index,  2671
predicted, actual 1 1.0
256
index,  2222
index,  2267
index,  2131
predicted, actual 1 1.0
257
index,  2460
index,  2849
index,  1073
predicted, actual 1 0.0
258
index,  442
index,  1985
index,  722
predicted, actual 0 1.0
259
index,  2532
index,  2333
index,  1093
predicted, actual 1 0.0
260
index,  324
index,  1132
index,  1095
predicted, actual 1 1.0
261
index,  2729
index,  2843
index,  739
predicted, actual 1 1.0
262
index,  1666
index,  1590
index,  2702
predicted, actual 1 1.0
263
index,  696
index,  438
index,  2754
predicted, actual 1 1.0
264
index,  622
index,  2250
index,  559
predicted, actual 1 1.0
265
index,  1918
index,  1294
index,  373
predicted, actual 1 1.0
266
index,  2760
index,  664
index,  1110
predicted, actual 1 1.0
267
index,

index,  1285
index,  1641
index,  666
predicted, actual 1 1.0
378
index,  939
index,  2828
index,  52
predicted, actual 1 1.0
379
index,  679
index,  705
index,  768
predicted, actual 1 1.0
380
index,  213
index,  2444
index,  2486
predicted, actual 1 1.0
381
index,  361
index,  2106
index,  540
predicted, actual 1 0.0
382
index,  1248
index,  1212
index,  2796
predicted, actual 1 1.0
383
index,  2781
index,  2836
index,  2897
predicted, actual 1 1.0
384
index,  1504
index,  1538
index,  517
predicted, actual 1 1.0
385
index,  455
index,  1565
index,  676
predicted, actual 1 1.0
386
index,  1702
index,  53
index,  1695
predicted, actual 1 1.0
387
index,  1000
index,  2505
index,  806
predicted, actual 1 1.0
388
index,  2024
index,  223
index,  3
predicted, actual 1 1.0
389
index,  834
index,  405
index,  2052
predicted, actual 1 1.0
390
index,  1710
index,  2758
index,  1974
predicted, actual 1 1.0
391
index,  512
index,  1567
index,  2012
predicted, actual 1 1.0
392
index,  153
index,

index,  97
index,  2865
index,  257
predicted, actual 1 1.0
503
index,  1372
index,  2361
index,  468
predicted, actual 1 1.0
504
index,  65
index,  92
index,  992
predicted, actual 1 1.0
505
index,  1305
index,  194
index,  2529
predicted, actual 1 1.0
506
index,  2309
index,  1411
index,  2551
predicted, actual 1 1.0
507
index,  97
index,  2865
index,  257
predicted, actual 1 1.0
508
index,  1323
index,  1458
index,  1011
predicted, actual 1 1.0
509
index,  1175
index,  2424
index,  1445
predicted, actual 1 1.0
510
index,  2506
index,  2373
index,  1205
predicted, actual 1 1.0
511
index,  949
index,  2327
index,  860
predicted, actual 1 0.0
512
index,  1912
index,  966
index,  775
predicted, actual 1 1.0
513
index,  1037
index,  128
index,  1655
predicted, actual 1 1.0
514
index,  1446
index,  404
index,  885
predicted, actual 1 1.0
515
index,  960
index,  2794
index,  381
predicted, actual 1 1.0
516
index,  1807
index,  2192
index,  312
predicted, actual 1 1.0
517
index,  2296
index

index,  2608
index,  534
index,  2599
predicted, actual 1 1.0
628
index,  1066
index,  1673
index,  1184
predicted, actual 1 1.0
629
index,  656
index,  90
index,  1231
predicted, actual 1 1.0
630
index,  637
index,  738
index,  2034
predicted, actual 1 1.0
631
index,  724
index,  2434
index,  2293
predicted, actual 1 1.0
632
index,  2180
index,  2011
index,  2112
predicted, actual 1 1.0
633
index,  2555
index,  677
index,  1214
predicted, actual 1 1.0
634
index,  2864
index,  2784
index,  962
predicted, actual 1 1.0
635
index,  2022
index,  2870
index,  1139
predicted, actual 1 1.0
636
index,  75
index,  1643
index,  1256
predicted, actual 1 1.0
637
index,  504
index,  945
index,  1046
predicted, actual 1 1.0
638
index,  2592
index,  1025
index,  1064
predicted, actual 1 1.0
639
index,  98
index,  1232
index,  1639
predicted, actual 1 1.0
640
index,  261
index,  1262
index,  2034
predicted, actual 1 1.0
641
index,  2854
index,  1631
index,  1315
predicted, actual 1 1.0
642
index,  208

In [16]:
print("Train set size: %d" % len(train_set))
print("Test set size: %d" % len(test_set))
print("Correct predicitons: %d" % correct)
print("Accuracy: %.2f%%" % (100 * correct / len(test_set)))

precision = tp/(tp + fp)
recall = tp/(tp + fn)
fscore = 2*((precision * recall)/(precision + recall))

print('precision: ', precision)
print('recall: ', recall)
print('fscore: ', fscore)

Train set size: 2916
Test set size: 730
Correct predicitons: 636
Accuracy: 87.12%
precision:  0.8948863636363636
recall:  0.9692307692307692
fscore:  0.930576070901034
